# 7.2.2. Multi Layer Perception Model

1. See Notion Notes for short-hand notation, explanations, my thoughts, etc.
2. BOOKS:
    - Deep Learning for Time Series Forecasting - Predict the Future with MLPs, CNNs and LSTMs in Python by Jason Brownlee
    - Introduction to TSF with Python - How to Prepare Data and Develop Models to Predict the Future by Jason Brownlee

# Imports + Load Data

In [1]:
from math import sqrt

import numpy as np
import pandas as pd


from keras.layers import Dense
from keras.models import Sequential
from sklearn.metrics import mean_squared_error

2023-05-29 17:51:59.474021: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
BASE = '/Users/brinkley97/Documents/development/'
PATH_TO_BOOK = 'book-intro_to_tsf_with_python/'

In [3]:
TS_AS_SML_FILE = BASE + PATH_TO_BOOK + '3-timeSeriesAsSupervisedLearning.ipynb'
TS_AS_SML_FILE

'/Users/brinkley97/Documents/development/book-intro_to_tsf_with_python/3-timeSeriesAsSupervisedLearning.ipynb'

In [4]:
# %load TS_AS_SML_FILE

In [5]:
# %run $TS_AS_SML_FILE

# For Full Datasets

In [6]:
dataset = BASE + PATH_TO_BOOK + 'datasets/daily-min-temperatures.csv'
series = pd.read_csv(dataset, header=0, index_col=0)
# series

In [7]:
data = series.values
# data

In [8]:
def train_test_split(data, n_test):
    
    train_X_y = data[:-n_test]
    print("train_X_y:", len(train_X_y), train_X_y)
    
    test_X_y = data[-n_test:]
    print("test_X_y:", len(test_X_y), test_X_y)
    
    return train_X_y, test_X_y

In [9]:
n_test = 12
train_data, test_data = train_test_split(data, n_test)

train_X_y: 3638 [[20.7]
 [17.9]
 [18.8]
 ...
 [13.9]
 [17.2]
 [14.7]]
test_X_y: 12 [[15.4]
 [13.1]
 [13.2]
 [13.9]
 [10. ]
 [12.9]
 [14.6]
 [14. ]
 [13.6]
 [13.5]
 [15.7]
 [13. ]]


In [10]:
# training_data

In [169]:
def convert_uts_sequence_to_sml(uts_observations, prior_observations, forecasting_step):
    """Splits a given UTS into multiple input rows where each input row has a specified number of timestamps and the output is a single timestamp.
    
    Parameters:
    uts_observations -- 1D np array (of UTS data to transform to SML data with size  b rows/length x 1 dimension)
    prior_observations -- py int (of all observations before we get to where we want to start making the predictions)
    forecasting_step -- py int (of how far out to forecast, 1 only the next timestamp, 2 the next two timestamps, ... n the next n timestamps)
    
    Return:
    agg.values -- np array (of new sml data)
    """
    
    df = pd.DataFrame(uts_observations)
    cols = list()
    
    lag_col_names = []
    count_lag = 0
    # input sequence (t-n, ... t-1)
    for prior_observation in range(prior_observations, 0, -1):
        # print("prior_observation: ", prior_observation)
        cols.append(df.shift(prior_observation))
        new_col_name = "t-" + str(prior_observation)
        # print(new_col_name)
        lag_col_names.append(new_col_name)
        
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, forecasting_step):
        cols.append(df.shift(-i))
        new_col_name = "t"
        # print(new_col_name)
        lag_col_names.append(new_col_name)
        
        # put it all together
        uts_sml_df = pd.concat(cols, axis=1) 
        uts_sml_df.columns=[lag_col_names]
        # drop rows with NaN values
        uts_sml_df.dropna(inplace=True)
    
    # print(uts_sml_df)
    
    # colums to use to make prediction for last col
    X_train = uts_sml_df.iloc[:, 0: -1]
    
    # last column
    y_train = uts_sml_df.iloc[:, [-1]]
    return X_train, y_train

In [174]:
prior_observations, forecasting_step = [24, 1]
X_train_df, y_train_df = convert_uts_sequence_to_sml(train_data, prior_observations, forecasting_step) 

In [175]:
X_train_df

,t-24,t-23,t-22,t-21,t-20,t-19,t-18,t-17,t-16,t-15,...,t-10,t-9,t-8,t-7,t-6,t-5,t-4,t-3,t-2,t-1
24,20.7,17.9,18.8,14.6,15.8,15.8,15.8,17.4,21.8,20.0,...,25.0,20.7,20.6,24.8,17.7,15.5,18.2,12.1,14.4,16.0
25,17.9,18.8,14.6,15.8,15.8,15.8,17.4,21.8,20.0,16.2,...,20.7,20.6,24.8,17.7,15.5,18.2,12.1,14.4,16.0,16.5
26,18.8,14.6,15.8,15.8,15.8,17.4,21.8,20.0,16.2,13.3,...,20.6,24.8,17.7,15.5,18.2,12.1,14.4,16.0,16.5,18.7
27,14.6,15.8,15.8,15.8,17.4,21.8,20.0,16.2,13.3,16.7,...,24.8,17.7,15.5,18.2,12.1,14.4,16.0,16.5,18.7,19.4
28,15.8,15.8,15.8,17.4,21.8,20.0,16.2,13.3,16.7,21.5,...,17.7,15.5,18.2,12.1,14.4,16.0,16.5,18.7,19.4,17.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3633,15.0,12.2,10.5,11.1,13.0,12.9,8.8,14.7,14.7,12.7,...,20.5,20.2,19.4,15.5,14.1,11.0,11.1,14.0,11.4,12.5
3634,12.2,10.5,11.1,13.0,12.9,8.8,14.7,14.7,12.7,13.3,...,20.2,19.4,15.5,14.1,11.0,11.1,14.0,11.4,12.5,13.4
3635,10.5,11.1,13.0,12.9,8.8,14.7,14.7,12.7,13.3,13.2,...,19.4,15.5,14.1,11.0,11.1,14.0,11.4,12.5,13.4,13.6
3636,11.1,13.0,12.9,8.8,14.7,14.7,12.7,13.3,13.2,16.2,...,15.5,14.1,11.0,11.1,14.0,11.4,12.5,13.4,13.6,13.9


In [176]:
y_train_df

,t
24,16.5
25,18.7
26,19.4
27,17.2
28,15.5
...,...
3633,13.4
3634,13.6
3635,13.9
3636,17.2


In [177]:
X_train = X_train_df.values[:, :-1]
print("X_train:", X_train)
y_train = y_train_df.values[:, -1] 
print("y_train", y_train)

X_train: [[20.7 17.9 18.8 ... 18.2 12.1 14.4]
 [17.9 18.8 14.6 ... 12.1 14.4 16. ]
 [18.8 14.6 15.8 ... 14.4 16.  16.5]
 ...
 [10.5 11.1 13.  ... 11.4 12.5 13.4]
 [11.1 13.  12.9 ... 12.5 13.4 13.6]
 [13.  12.9  8.8 ... 13.4 13.6 13.9]]
y_train [16.5 18.7 19.4 ... 13.9 17.2 14.7]


In [15]:
# fit a model 
def mlp_model_fit(X_train, y_train, n_nodes, n_epochs, n_batch, prior_observations): 
    """
    Parameters: 
    sml_data -- 
    
    """
    # define model
    model = Sequential() 
    model.add(Dense(n_nodes, activation='relu' , input_dim=prior_observations)) 
    model.add(Dense(1)) 
    model.compile(loss='mse' , optimizer='adam') 
    
    # fit
    model.fit(X_train, y_train, epochs=n_epochs, batch_size=n_batch, verbose=0) 
    
    return model

In [16]:
# n_nodes, n_epochs, n_batch = [500, 100, 100] 
# print(n_epochs)
# model = mlp_model_fit(converted_uts_to_sml, n_nodes, n_epochs, n_batch, prior_observations) 

In [17]:
# root mean squared error or rmse
def measure_rmse(actual, predicted):
    # print("ACTUAL = ", actual)
    # print("PREDICTED = ", predicted)
    return sqrt(mean_squared_error(actual, predicted))

In [18]:
# forecast with a pre-fit model 
def model_predict(model, history, n_input): 

    x_input = np.array(history[-n_input:]).reshape(1, n_input) 

    # forecast 
    yhat = model.predict(x_input, verbose=0) 

    
    return yhat[0]

In [19]:
# walk-forward validation for univariate data 
def walk_forward_validation(test_data, model, history, prior_observations, predictions): 

    # step over each time-step in the test set 
    for i in range(len(test_data)): 
        # print(history)
        # fit model and make forecast for history 
        yhat = model_predict(model, history, prior_observations) 
        # print(yhat)
        
        # store forecast in list of predictions 
        predictions.append(yhat)
        # print(i, predictions)
        
        # add actual observation to history for the next loop 
        # print(test_data[i])
        history.append(test_data[i])
    # print(predictions)
    # estimate prediction error 
    error = measure_rmse(test_data, predictions) 
    # print('> %.3f' % error) 
    
    return error

In [20]:
# walk_forward_validation(train_data, test_data, model, prior_observations)

In [21]:
# repeat evaluation of a config 
def repeat_evaluate(X_train, y_train, test_data, n_test, n_nodes, n_epochs, n_batch, n_repeats, prior_observations):

    store_scores = []

    for i in range(n_repeats):
        print(i)
        predictions = list()
        model = mlp_model_fit(X_train, y_train, n_nodes, n_epochs, n_batch, prior_observations) 
 
    
        history = [x for x in train_data]
        
        score = walk_forward_validation(test_data, model, history, prior_observations, predictions)
        print("score is : ", score, "for index", i)
        store_scores.append(score)
        print()
        
    return store_scores
    # return scores

In [22]:
n_nodes, n_epochs, n_batch = [12, 500, 100] 
n_repeats = 3

In [23]:
scores = repeat_evaluate(X_train, y_train, test_data, n_test, n_nodes, n_epochs, n_batch, n_repeats, prior_observations)

0


2023-05-29 17:52:06.368771: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


score is :  1.6718007487223239 for index 0

1
score is :  1.6890238428438575 for index 1

2
score is :  1.7217974025691891 for index 2



In [24]:
# # summarize model performance
# def summarize_scores(name, scores): 
#     # print a summary
#     scores_m, score_std = mean(scores), std(scores)
#     print(' %s: %.3f RMSE (+/- %.3f)' % (name, scores_m, score_std)) 
    
    # box and whisker plot 
    # pyplot.boxplot(scores)
    # pyplot.show()

In [25]:
# summarize_scores('mlp' , scores)

In [26]:
# train, test = split_data(data)

In [27]:
# config = [24, 500, 100, 100] 
# model = model_fit(train, cfg) 

In [28]:
# # define config
# config = [24, 500, 100, 100] 

# # grid search 
# scores = repeat_evaluate(data, config, n_test)
scores = repeat_evaluate(training_data, n_test, model, prior_observations, test_data)

NameError: name 'training_data' is not defined

In [ ]:
# summarize scores 
